# Comparing legislation

In this notebook I use a language model to compare pieces of legislation.

In [1]:
!python --version

Python 3.9.17


In [2]:
!openai --version

openai 0.27.10


In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Load the documents

In [4]:
from pydantic import BaseModel, Field

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

In [5]:
path_to_popi_act = "/Users/ekowduker/Library/Mobile Documents/com~apple~CloudDocs/Desktop/The AI Shop/comparing_legislation/protection_of_personal_information.pdf"
path_to_cybercrime_act = "/Users/ekowduker/Library/Mobile Documents/com~apple~CloudDocs/Desktop/The AI Shop/comparing_legislation/cybercrime.pdf"

In [6]:
class DocumentInput(BaseModel):
    question: str = Field()


llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

tools = []
files = [
    # Protection of Personal Information Act 2013
    {
        "name": "protection_of_personal_information_act",
        "path": path_to_popi_act,
    },
    # Cybercrimes Act 2020
    {
        "name": "cybercrimes_act",
        "path": path_to_cybercrime_act,
    },
]

for file in files:
    loader = PyPDFLoader(file["path"])
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    embeddings = OpenAIEmbeddings()
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()

    # Wrap retrievers in a Tool
    tools.append(
        Tool(
            args_schema=DocumentInput,
            name=file["name"],
            description=f"useful when you want to answer questions about {file['name']}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
        )
    )

In [7]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [8]:
llm = ChatOpenAI(
    temperature=0,
    model = "gpt-4-1106-preview"
)

agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)

In [9]:
answer = agent({"input": f"what is the popi act about?"})
print(answer['output'])



> Entering new AgentExecutor chain...

Invoking: `protection_of_personal_information_act` with `{'question': 'What is the POPI Act about?'}`


{'query': 'What is the POPI Act about?', 'result': 'The Protection of Personal Information Act (POPI Act), Act No. 4 of 2013, is a South African law designed to promote the protection of personal information processed by public and private bodies. It introduces conditions to establish minimum requirements for the processing of personal information and provides for the establishment of an Information Regulator to exercise certain powers and perform certain duties and functions in terms of this Act and the Promotion of Access to Information Act, 2000.\n\nThe Act aims to regulate the processing of personal information in harmony with international standards, ensuring that the right to privacy is respected while also considering the need for economic and social progress within the information society. It addresses the lawful processing of personal

In [8]:
answer = agent({"input": f"give me a detailed summary of the popi act"})
print(answer['output'])



> Entering new AgentExecutor chain...
The Protection of Personal Information Act (POPIA) is South Africa's data protection law that governs the processing of personal information. It was signed into law on November 19, 2013, and parts of the Act have been put into effect incrementally, with the latest sections coming into force on July 1, 2020. The Act aims to promote the protection of personal information processed by public and private bodies and to establish minimum requirements for the processing of such information.

Here is a detailed summary of the key aspects of the POPI Act:

1. **Scope and Application**: The POPI Act applies to the processing of personal information entered into a record by or for a responsible party by making use of automated or non-automated means. It applies to those within South Africa and sometimes to parties outside South Africa if they process personal information within the country.

2. **Conditions for Lawful Processing**: The Act sets out eight co

In [9]:
answer = agent({"input": "what is the cybercrime act about?"})
print(answer['output'])



> Entering new AgentExecutor chain...

Invoking: `cybercrimes_act` with `{'question': 'What is the Cybercrimes Act about?'}`


{'query': 'What is the Cybercrimes Act about?', 'result': 'The Cybercrimes Act, 2020 is a South African law that aims to address cybercrime and protect against harmful data messages. It creates offenses related to cybercrime, criminalizes the disclosure of harmful data messages, and provides for interim protection orders. The Act also regulates jurisdiction in cybercrime cases, powers to investigate cybercrimes, mutual assistance in investigations, and the establishment of a designated Point of Contact. It imposes obligations to report cybercrimes, provides for capacity building, and allows the Executive to enter into agreements with foreign states to combat cybercrimes. The Act also includes provisions for the deletion and amendment of certain laws related to cybercrimes.'}The Cybercrimes Act, 2020 is a South African law designed to combat cybercrime and pro

In [10]:
answer = agent({"input": f"give me a detailed summary of the cybercrime act"})
print(answer['output'])



> Entering new AgentExecutor chain...

Invoking: `cybercrimes_act` with `{'question': 'Can you provide a detailed summary of the Cybercrimes Act?'}`


{'query': 'Can you provide a detailed summary of the Cybercrimes Act?', 'result': 'The Cybercrimes Act, 2020 is a piece of legislation enacted by the Parliament of the Republic of South Africa. It aims to address cybercrime and create offenses related to cybercrime. The Act criminalizes the disclosure of harmful data messages and provides for interim protection orders. It also regulates jurisdiction in relation to cybercrimes and the powers to investigate such crimes.\n\nThe Act covers various aspects of cybercrime, including unlawful access, unlawful interception of data, unlawful acts in respect of software or hardware tools, and unlawful interference with data or computer programs. It also addresses malicious communications and provides for sentencing and orders to protect complainants from the harmful effects of malicious communica

In [11]:
areas_of_overlap = f"are there any areas of overlap between the two pieces of legislation?"
answer = agent({"input": areas_of_overlap})
print(answer['output'])



> Entering new AgentExecutor chain...

Invoking: `protection_of_personal_information_act` with `{'question': 'Does the Protection of Personal Information Act cover any aspects of cybercrimes or cyber security?'}`


{'query': 'Does the Protection of Personal Information Act cover any aspects of cybercrimes or cyber security?', 'result': 'Based on the provided context, there is no specific mention of cybercrimes or cyber security in the Protection of Personal Information Act. Therefore, it is unclear whether the act covers these aspects.'}
Invoking: `cybercrimes_act` with `{'question': 'Does the Cybercrimes Act address the protection of personal information?'}`


{'query': 'Does the Cybercrimes Act address the protection of personal information?', 'result': 'Yes, the Cybercrimes Act does address the protection of personal information. It references the "Protection of Personal Information Act, 2013" and states that any failure by a responsible party to comply with the conditions for law

In [12]:
areas_of_contradiction = f"Are there any areas of contradiction between the \
two pieces of legislation?"
answer = agent({"input": areas_of_contradiction})
print(answer['output'])



> Entering new AgentExecutor chain...

Invoking: `protection_of_personal_information_act` with `{'question': 'Are there any contradictions between the Protection of Personal Information Act and the Cybercrimes Act?'}`


{'query': 'Are there any contradictions between the Protection of Personal Information Act and the Cybercrimes Act?', 'result': 'Based on the provided context, there is no mention of the Cybercrimes Act. Therefore, it is not possible to determine if there are any contradictions between the Protection of Personal Information Act and the Cybercrimes Act.'}
Invoking: `cybercrimes_act` with `{'question': 'Are there any contradictions between the Cybercrimes Act and the Protection of Personal Information Act?'}`


{'query': 'Are there any contradictions between the Cybercrimes Act and the Protection of Personal Information Act?', 'result': 'Yes, there are no contradictions between the Cybercrimes Act and the Protection of Personal Information Act. In fact, the Cybercrimes 

#### OpenAI Multifunctions

This type of agent allows calling multiple functions at once. This is really useful when some steps can be computed in parallel - like when asked to compare multiple documents

In [13]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
)

agent = initialize_agent(
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
)

In [14]:
answer = agent({"input": f"what is the popi act about?"})
print(answer['output'])



> Entering new AgentExecutor chain...
The Protection of Personal Information Act (POPI Act) is a South African law that aims to protect the privacy and personal information of individuals. It regulates how personal information is collected, processed, stored, and shared by public and private organizations. The POPI Act establishes principles and conditions for the lawful processing of personal information, and it grants individuals certain rights and remedies in relation to their personal information. The act also imposes obligations on organizations to ensure the security and confidentiality of personal information.

> Finished chain.
The Protection of Personal Information Act (POPI Act) is a South African law that aims to protect the privacy and personal information of individuals. It regulates how personal information is collected, processed, stored, and shared by public and private organizations. The POPI Act establishes principles and conditions for the lawful processing of pers

In [15]:
answer = agent({"input": f"give me a detailed summary of the popi act"})
print(answer['output'])



> Entering new AgentExecutor chain...
The Protection of Personal Information Act (POPIA), also known as the POPI Act, is a South African law that aims to protect the privacy and personal information of individuals. It was signed into law in 2013 and came into effect on July 1, 2020.

The main purpose of the POPI Act is to regulate the processing of personal information by public and private bodies. It establishes certain principles and conditions that organizations must adhere to when collecting, storing, and processing personal information.

Here are some key points and provisions of the POPI Act:

1. Definitions: The act provides definitions for various terms related to personal information, such as data subject, responsible party, operator, and processing.

2. Conditions for lawful processing: The act sets out conditions that must be met for the lawful processing of personal information. These conditions include obtaining the consent of the data subject, processing for a legitimat

In [16]:
answer = agent({"input": f"what is the cybercrime act about?"})
print(answer['output'])



> Entering new AgentExecutor chain...
The Cybercrimes Act is a legislation that aims to address and prevent cybercrimes. It provides legal provisions and guidelines for investigating, prosecuting, and punishing individuals or groups involved in cybercrimes. The act covers a wide range of offenses, including hacking, identity theft, online fraud, cyberbullying, and the distribution of malicious software. It also outlines the procedures for collecting electronic evidence and cooperation between law enforcement agencies and internet service providers. The Cybercrimes Act is designed to protect individuals, organizations, and the overall security of cyberspace.

> Finished chain.
The Cybercrimes Act is a legislation that aims to address and prevent cybercrimes. It provides legal provisions and guidelines for investigating, prosecuting, and punishing individuals or groups involved in cybercrimes. The act covers a wide range of offenses, including hacking, identity theft, online fraud, cyb

In [17]:
answer = agent({"input": f"give me a detailed summary of the cybercrime act"})
print(answer['output'])



> Entering new AgentExecutor chain...
The Cybercrimes Act is a legislation that aims to address and combat cybercrimes in a country. It provides a legal framework to prevent, investigate, and prosecute offenses related to computer systems, networks, and electronic communications. Here is a detailed summary of the Cybercrimes Act:

1. Scope and Definitions:
   - The act defines various terms related to cybercrimes, such as computer, computer system, data, electronic communication, hacking, identity theft, phishing, etc.
   - It specifies the scope of the act, which generally covers offenses committed using computer systems, networks, and electronic communications.

2. Offenses and Penalties:
   - The act lists various offenses related to cybercrimes, including unauthorized access to computer systems, data interference, system interference, cyber fraud, cyber stalking, cyber terrorism, etc.
   - It defines the penalties for each offense, which may include fines, imprisonment, or both, 

In [18]:
answer = agent({"input": "do the acts contradict each other in any way?"})
print(answer['output'])



> Entering new AgentExecutor chain...
The Protection of Personal Information Act (POPIA) and the Cybercrimes Act do not directly contradict each other. However, there may be some areas where the two acts overlap or have different requirements that could potentially create some challenges or confusion in implementation. It is important to carefully review both acts and consult legal experts to ensure compliance with all relevant laws and regulations.

> Finished chain.
The Protection of Personal Information Act (POPIA) and the Cybercrimes Act do not directly contradict each other. However, there may be some areas where the two acts overlap or have different requirements that could potentially create some challenges or confusion in implementation. It is important to carefully review both acts and consult legal experts to ensure compliance with all relevant laws and regulations.


In [19]:
answer = agent({"input": "do the acts overlap in any way?"})
print(answer['output'])



> Entering new AgentExecutor chain...
The Protection of Personal Information Act (POPIA) and the Cybercrimes Act do have some areas of overlap. Both acts aim to protect individuals' personal information and address issues related to data privacy and security. However, they have different focuses and cover different aspects of these issues.

POPIA primarily focuses on the protection of personal information and sets out principles and conditions for the lawful processing of personal information. It establishes the rights of individuals regarding their personal information and imposes obligations on organizations that collect, store, and process personal information.

On the other hand, the Cybercrimes Act focuses on addressing cybercrimes and provides for the investigation, prevention, and prosecution of various cyber offenses. It covers a wide range of cybercrimes, including unauthorized access to computer systems, data interference, computer-related fraud, and cyberbullying.

While t

Housekeeping

In [1]:
#!pip install --upgrade voila


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
#!pip install --upgrade ipywidgets

  Using cached ipywidgets-8.1.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.9-py3-none-any.whl.metadata (1.6 kB)
Using cached ipywidgets-8.1.1-py3-none-any.whl (139 kB)
Using cached widgetsnbextension-4.0.9-py3-none-any.whl (2.3 MB)
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.5.2
    Uninstalling widgetsnbextension-3.5.2:
      Successfully uninstalled widgetsnbextension-3.5.2
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.6.5
    Uninstalling ipywidgets-7.6.5:
      Successfully uninstalled ipywidgets-7.6.5

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
